In [1]:
# import python file from parent folder
from img_cap_lib import *
# imports
import torch
import torchvision
import torchtext
from torchtext.vocab import vocab, GloVe, Vectors
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import os
from PIL import Image
import string
from collections import OrderedDict, Counter
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
import pickle
import os
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import nltk

# Daten herunterladen

In [2]:
data_download("flickr8k")

Data already exi sts at flickr8k


# Preprocessing

In [3]:
# caption preprocessing
embedding_dim = 300
min_frequency = 1

captions = pd.read_csv("flickr8k/captions.txt")
caption_preprocessor = CaptionPreprocessor(captions=captions, embedding_dim=embedding_dim, min_frequency=min_frequency)
caption_preprocessor.preprocess()

# image preprocessing
img_preprocessor = ImagePreprocessor(normalize=True, image_folder_path="flickr8k")
img_preprocessor.preprocess_images()

Shape captions: (40460, 2)


/Users/ronnyschneeberger/Documents/FHNW/HS22/del-image-captioning/img_cap_lib.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.captions.caption = self.captions.caption.apply(lambda x: x.strip("."))


Shape captions after filtering: (39749, 3)
Removed Captions:  711 , in Percent:  1.76
transformed_images folder already exists. No preprocessing necessary.


# Data Split

In [4]:
batch_size = 64

training_data, test_data = train_test_split(caption_preprocessor.captions, test_size=0.15, random_state=42)

embedding = Embedding(embedding_matrix=caption_preprocessor.embedding, vocabulary=caption_preprocessor.vocabulary)

# create dataset
train_dataset = FlickrDataset(captions=training_data, embedding=embedding)
test_dataset = FlickrDataset(captions=test_data, embedding=embedding)

# create dataloader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

# Modell erstellen

In [5]:
if "standart.pt" in os.listdir():
    model_stats = torch.load("standart.pt")
    model = load_captioning_model(model_stats)
else:
    encoder = EncoderCNN(net=torchvision.models.resnext50_32x4d, pretrained_weights=torchvision.models.ResNeXt50_32X4D_Weights.IMAGENET1K_V2, output_size=300)
    decoder = DecoderRNN(input_size=300, hidden_size=caption_preprocessor.embedding_dim, num_layers=1, dropout=0.0, len_vocab=embedding.embedding_matrix.shape[0], len_subtract=0)
    model = ImageCaptioning(encoder=encoder, decoder=decoder, embedding=embedding, batch_size=batch_size)

In [5]:
temp_model_stats = torch.load("models/temp_model.pt", map_location=torch.device(device))
model = load_captioning_model(temp_model_stats)

model.train_model(loader=train_loader, optimizer=torch.optim.Adam(model.parameters(), lr=0.01), criterion=torch.nn.CrossEntropyLoss(), epochs=1, print_every=1)

Epoch: 1/1 | Batch: 1/527 | Loss: 1.29318368434906
Epoch: 1/1 | Batch: 2/527 | Loss: 1.9721699953079224
Epoch: 1/1 | Batch: 3/527 | Loss: 2.5652785301208496
Epoch: 1/1 | Batch: 4/527 | Loss: 2.14449143409729
Epoch: 1/1 | Batch: 5/527 | Loss: 3.046929359436035
Epoch: 1/1 | Batch: 6/527 | Loss: 2.757331371307373
Epoch: 1/1 | Batch: 7/527 | Loss: 2.5614356994628906
Epoch: 1/1 | Batch: 8/527 | Loss: 2.1697428226470947
Epoch: 1/1 | Batch: 9/527 | Loss: 2.110513687133789
Epoch: 1/1 | Batch: 10/527 | Loss: 2.930253505706787
Epoch: 1/1 | Batch: 11/527 | Loss: 2.250471591949463
Epoch: 1/1 | Batch: 12/527 | Loss: 2.630889654159546
Epoch: 1/1 | Batch: 13/527 | Loss: 2.4383749961853027
Epoch: 1/1 | Batch: 14/527 | Loss: 2.949169874191284
Epoch: 1/1 | Batch: 15/527 | Loss: 2.97184157371521
Epoch: 1/1 | Batch: 16/527 | Loss: 2.0929834842681885
Epoch: 1/1 | Batch: 17/527 | Loss: 2.8149142265319824
Epoch: 1/1 | Batch: 18/527 | Loss: 2.2896153926849365
Epoch: 1/1 | Batch: 19/527 | Loss: 2.4711496829986

KeyboardInterrupt: 

In [ ]:
# train model
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
new_model_stats = model.train_model(loader=train_loader, optimizer=optimizer, criterion=criterion, epochs=200, print_every=1000)

if model_stats is not None:
    new_model_stats['epochs'] += model_stats['epochs']
    new_model_stats['total_time'] += model_stats['total_time']
    new_model_stats['losses'] = model_stats['losses'] + new_model_stats['losses']
    new_model_stats['average_losses'] = model_stats['average_losses'] + new_model_stats['average_losses']

torch.save(model_stats, "standart.pt")

# load temp model
temp_model_stats = torch.load("temp_model.pt", map_location=torch.device(device))

# compare losses
if temp_model_stats['last_loss'] < model_stats['last_loss']:
    # replace model structure
    for key in temp_model_stats.keys():
        model_stats[key] = temp_model_stats[key]
    torch.save(model_stats, "standart.pt")

Epoch: 1/250 | Batch: 1/527 | Loss: 9.025182723999023
Epoch: 1/250 | Average Epoch Loss: 5.990393426884284
Epoch: 2/250 | Batch: 1/527 | Loss: 5.4601640701293945
Epoch: 2/250 | Average Epoch Loss: 5.229636007739652
Epoch: 3/250 | Batch: 1/527 | Loss: 5.013434886932373
Epoch: 3/250 | Average Epoch Loss: 4.777546999350444
Epoch: 4/250 | Batch: 1/527 | Loss: 4.598186492919922
Epoch: 4/250 | Average Epoch Loss: 4.37486219225165
Epoch: 5/250 | Batch: 1/527 | Loss: 4.058480739593506
Epoch: 5/250 | Average Epoch Loss: 4.002896713350936
Epoch: 6/250 | Batch: 1/527 | Loss: 3.611685276031494
Epoch: 6/250 | Average Epoch Loss: 3.6396884777966667
Epoch: 7/250 | Batch: 1/527 | Loss: 3.206207036972046
Epoch: 7/250 | Average Epoch Loss: 3.2965542238622736
Epoch: 8/250 | Batch: 1/527 | Loss: 3.3456461429595947
Epoch: 8/250 | Average Epoch Loss: 2.9615128452004247
Epoch: 9/250 | Batch: 1/527 | Loss: 2.731268882751465
Epoch: 9/250 | Average Epoch Loss: 2.6515791284970134
Epoch: 10/250 | Batch: 1/527 | L

# Evaluierung

In [ ]:
class Evaluator:
    def __init__(self, model, dataloader, device):
        # initiate variables 
        self.model = model
        self.dataloader = dataloader
        self.device = device
        # self.model.eval()
        # assert self.dataloader.batch_size == 1, "Batch size must be 1 for evaluation."
    
    def evaluate(self):
        scores = []

        for i, (images, captions, lengths, vectorized_captions) in enumerate(self.dataloader):
            # move to device
            images = images.to(self.device)
            captions = captions.to(self.device)
            vectorized_captions = vectorized_captions.to(self.device)
            
            # forward pass
            output = self.model.forward(images)
            references = self.model.words[vectorized_captions.cpu()]

            for j in range(output.shape[0]):
                candidate = self.output_to_sentence(output[j,:])
                reference = self.output_to_sentence(references[j,:])
                scores.append(self.bleu_score(candidate, reference))
            
            print(f"Batch: {i+1} of {len(self.dataloader)}")

        print(f"Average BLEU score: {np.mean(scores)}")
        return np.mean(scores), scores

    @staticmethod
    def output_to_sentence(output:list):
        '''
        Removes Tokens from model output.
        '''
        output = [token for token in output if token not in ["<SOS>", "<EOS>", "<PAD>"]]
        return output

    @staticmethod
    def bleu_score(reference, candidate):
        '''
        Calculates the BLEU score for a single reference and candidate. Uses the SmoothingFunction for smoothing when no overlap between certain n-grams is found. 

        Params:
        -------
        reference: list of strings - The reference sentence.
        candidate: list of strings - The candidate sentence.

        Returns:
        --------
        bleu_score: float - The BLEU score.
        '''
        # calculate the BLEU score
        return nltk.translate.bleu_score.sentence_bleu(reference, candidate, smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method1)

In [ ]:
path = 'standart.pt'
model_stats = torch.load(path, map_location=device)
model = load_captioning_model(model_stats)

In [ ]:
train_dataset = FlickrDataset(captions=training_data, embedding=embedding)
test_dataset = FlickrDataset(captions=test_data, embedding=embedding)

# create dataloader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True, drop_last=True)

# calculate bleu scores
train_evaluator = Evaluator(model, train_loader, device)
test_evaluator = Evaluator(model, test_loader, device)

train_bleu, train_scores = train_evaluator.evaluate()
test_bleu, test_scores = test_evaluator.evaluate()

print(f"Train BLEU: {train_bleu}")
print(f"Test BLEU: {test_bleu}")

In [ ]:
# export bleu scores
with open("standart_train_scores.pkl", "wb") as f:
    pickle.dump(train_scores, f)

with open("standart_train_scores.pkl", "wb") as f:
    pickle.dump(test_scores, f)